# What is a View?
- it is not a real table but a database object that stores a query.
- representation of your data, does not store anything unless you are using a 'materialized view'.
- can be used for subsetting data, joining multiple tables, even granting permissions are an option

++materialized views: very fast lookups and often used in data-warehousing 
++ you cannot used CREATE VIEW IF NOT EXISTS, use instead CREATE OR REPLACE VIEW

In [6]:
import psycopg2           
import pandas as pd   
import seaborn as sns
from config import config# Import the 'config' function from the config.py file:

# Get the config params
params_ = config()
# Connect to the Postgres_DB:
conn = psycopg2.connect(**params_)
# Create new_cursor allowing us to write Python to execute PSQL:
cur = conn.cursor()
conn.autocommit = True

In [7]:
sql_="SELECT * FROM  ppl_cpu_purchases LIMIT 2"
cur.execute(sql_)
cur.fetchall()

# pd.DataFrame(cur.fetchall(),columns=['credit_card','email','first_name','last_name',
# 'primary_phone','cpu','purchase_date'])

[('5399-3484-4724-7187',
  'gso@qiegan.sqe',
  'Donyell Ann',
  'Ospina',
  '5219459148',
  'Intel Core i1-9809K',
  datetime.date(2020, 3, 26)),
 ('1630-5261-6108-7631',
  'xnji@gfruaxqnvm.fha',
  'Bishop',
  'Siyed',
  '4164254716',
  'AMD Ryzen 2 7889X',
  datetime.date(2017, 12, 10))]

View  1<br>
- showing first,last name with credit card number and the cpu with purchase date

In [32]:
# Create View
sql_view=""" CREATE VIEW customer_cpu_sales AS
SELECT staging_fake_ppl.first_name || ' ' || staging_fake_ppl.last_name AS full_name, staging_fake_ppl.credit_card AS card,
staging_fake_cpu_purchases.cpu,staging_fake_cpu_purchases.purchase_date
FROM staging_fake_cpu_purchases 
INNER JOIN staging_fake_ppl 
ON staging_fake_ppl.credit_card=staging_fake_cpu_purchases.credit_card"""

cur.execute(sql_view)

In [33]:
# QUERY OUR first VIEW:

s = "SELECT * FROM customer_cpu_sales LIMIT 4"
cur.execute(s)
cur.fetchall()

[('Nasreen Angel',
  '1000-1293-3496-7245',
  'AMD Ryzen 5 8206X',
  datetime.date(2018, 10, 21)),
 ('Qirsten el-Azimi',
  '1000-7698-8868-7669',
  'Intel Core i2-4914K',
  datetime.date(2020, 8, 3)),
 ('Alejandro Batchelor',
  '1002-7633-3301-5737',
  'Intel Core i3-6772K',
  datetime.date(2018, 3, 12)),
 ('Gabriel Salazar',
  '1003-1072-2532-5536',
  'AMD Ryzen 8 3128X',
  datetime.date(2020, 10, 17))]

In [25]:
# Update View: for example change the (name of our VIEW)
# cur.execute("ALTER VIEW customer_cpu_sales RENAME TO customer_info")

In [26]:
# Delete View:
# cur.execute("DROP VIEW customer_info")

In [28]:
# Print no. of days since purchase
sales_byDate="select date_trunc('day',now() - purchase_date::date) from customer_cpu_sales"
cur.execute(sales_byDate)
cur.fetchone()

(datetime.timedelta(days=209),)

View 2
- create a view of days elapsed since purchase tracked from today

In [54]:
cust_warr_view="CREATE OR REPLACE VIEW customer_warranty AS SELECT *,\
date_trunc('day',now() - purchase_date::date) \
AS days_since_purchase FROM ppl_cpu_purchases"
cur.execute(cust_warr_view)

sqq="select * from customer_warranty limit 2"
cur.execute(sqq)
cur.fetchall()

[('5399-3484-4724-7187',
  'gso@qiegan.sqe',
  'Donyell Ann',
  'Ospina',
  '5219459148',
  'Intel Core i1-9809K',
  datetime.date(2020, 3, 26),
  datetime.timedelta(days=209)),
 ('1630-5261-6108-7631',
  'xnji@gfruaxqnvm.fha',
  'Bishop',
  'Siyed',
  '4164254716',
  'AMD Ryzen 2 7889X',
  datetime.date(2017, 12, 10),
  datetime.timedelta(days=1046))]

In [56]:
dates_n_warr="SELECT *,\
date_trunc('day',now() - purchase_date::date) \
AS days_since_purchase, \
CASE WHEN extract(day from purchase_date::date) < 180 \
THEN 'Yes' ELSE 'No' END AS warranty FROM ppl_cpu_purchases LIMIT 2"

cur.execute(dates_n_warr)
cur.fetchall()


[('5399-3484-4724-7187',
  'gso@qiegan.sqe',
  'Donyell Ann',
  'Ospina',
  '5219459148',
  'Intel Core i1-9809K',
  datetime.date(2020, 3, 26),
  datetime.timedelta(days=209),
  'Yes'),
 ('1630-5261-6108-7631',
  'xnji@gfruaxqnvm.fha',
  'Bishop',
  'Siyed',
  '4164254716',
  'AMD Ryzen 2 7889X',
  datetime.date(2017, 12, 10),
  datetime.timedelta(days=1046),
  'Yes')]

In [57]:
# Finding column names
cur.execute("Select * FROM ppl_cpu_purchases LIMIT 0")
colnames = [desc[0] for desc in cur.description]
colnames

['credit_card',
 'email',
 'first_name',
 'last_name',
 'primary_phone',
 'cpu',
 'purchase_date']

In [29]:
# Create Query
# Monthly Sales of CPU

cpu_g="SELECT pg_catalog.date(date_trunc('month',purchase_date)) AS txn_month, \
count(cpu) as monthly_sum \
FROM ppl_cpu_purchases GROUP BY txn_month ORDER BY txn_month DESC"

cur.execute(cpu_g)
pd.DataFrame(cur.fetchall(),columns=['Month_yr','Items Sold']).head(7)




,Month_yr,Items Sold
0,2020-10-01,129
1,2020-09-01,149
2,2020-08-01,157
3,2020-07-01,166
4,2020-06-01,138
5,2020-05-01,181
6,2020-04-01,151
